# Imports

In [1]:
import pandas as pd  # Importa a biblioteca pandas para manipulação e análise de dados.
import requests  # Importa a biblioteca requests para fazer requisições HTTP.
import base64  # Importa a biblioteca base64 para codificação e decodificação de dados em Base64.
from IPython.display import display, Javascript  # Importa funções para exibir resultados e executar JavaScript no IPython.

# Access token requisition

## Variables

In [2]:
client_id = 'c6ca0e7ef3d541d187ea9bbda30f9304'  # Define o ID do cliente do Spotify como uma string.
client_secret = '29127565b24644959312ba8c3eab8c8e'  # Define o segredo do cliente do Spotify como uma string.

## Base 64 conversion

In [3]:
string = client_id + ':' + client_secret  # Concatena o client_id e o client_secret com um ':' no meio.
string_bytes = string.encode('ascii')  # Codifica a string concatenada em bytes ASCII.
base64_bytes = base64.b64encode(string_bytes)  # Codifica os bytes ASCII em Base64.
base64_string = base64_bytes.decode('ascii')  # Decodifica os bytes Base64 para uma string ASCII.

## Requisition

In [4]:
url = 'https://accounts.spotify.com/api/token'  # Define a URL do endpoint para obter o token de acesso do Spotify.
headers = {
    'Authorization': f'Basic {base64_string}',  # Define o cabeçalho de autorização utilizando a string codificada em Base64.
    'Content-Type': 'application/x-www-form-urlencoded'  # Define o tipo de conteúdo como URL codificada.
}
payload = {'grant_type': 'client_credentials'}  # Define o payload com o tipo de concessão para credenciais de cliente.
response = requests.request('POST', url=url, headers=headers, data=payload)  # Faz uma requisição POST para a URL com os cabeçalhos e o payload definidos.
access_token = response.json()['access_token']  # Extrai o token de acesso da resposta JSON.

# Artists dataframe

In [9]:
def show_alert(message):
    display(Javascript(f'alert("{message}")'))  # Função para exibir um alerta JavaScript com a mensagem fornecida.

def get_artist_data(artist_id, access_token):
    url = f'https://api.spotify.com/v1/artists/{artist_id}'  # Define a URL para obter dados de um artista específico pelo ID.
    headers = {'Authorization': f'Bearer {access_token}'}  # Define o cabeçalho de autorização com o token de acesso.
    response = requests.get(url, headers=headers)  # Faz uma requisição GET para a URL com os cabeçalhos definidos.

    if response.status_code != 200:  # Verifica se a resposta não é bem-sucedida (código diferente de 200).
        show_alert(f'Error {response.status_code}: {response.json().get("error", {}).get("message", "Erro ao alimentar a tabela df_artist.")}')
        return None  # Exibe um alerta com a mensagem de erro e retorna None.
    return response.json()  # Retorna os dados do artista em formato JSON.

artist_ids = [
    '10gzBoINW3cLJfZUka8Zoe', '4q3ewBCX7sLwd24euuV69X',
    '25uiPmTg16RbhZWAqwLBy5', '6M2wZ9GZgrQXHCFfjv46we', '0X2BH1fck6amBIoJhDVmmJ'
]  # Lista de IDs de artistas do Spotify.

artist_data = []
for artist_id in artist_ids:
    data = get_artist_data(artist_id, access_token)  # Obtém os dados de cada artista usando o ID e o token de acesso.
    if data is not None:
        artist_data.append(data)  # Adiciona os dados do artista à lista se não forem None.

# Criar uma lista para armazenar os dados dos artistas
artists_list = []

for artist in artist_data:
    genres_list = artist['genres']
    for genre in genres_list:
        artists_list.append({
            'id': str(artist['id']),  # Garante que o id seja uma string.
            'name': str(artist['name']),  # Garante que o name seja uma string.
            'genres': str(genre),  # Garante que o genres seja uma string.
            'popularity': int(artist['popularity']),  # Converte popularity para inteiro.
            'followers': int(artist['followers']['total'])  # Converte followers para inteiro.
        })

# Cria um DataFrame a partir da lista de artistas
df_artists = pd.DataFrame(artists_list)

# Salvar o DataFrame em um arquivo CSV
df_artists.to_csv('artists_data.csv', index=False, header=False)  # header=False remove os nomes das colunas do arquivo CSV

df_artists  # Exibe o DataFrame final com os dados dos artistas.

,id,name,genres,popularity,followers
0,10gzBoINW3cLJfZUka8Zoe,Above & Beyond,edm,60,798773
1,10gzBoINW3cLJfZUka8Zoe,Above & Beyond,progressive house,60,798773
2,10gzBoINW3cLJfZUka8Zoe,Above & Beyond,progressive trance,60,798773
3,10gzBoINW3cLJfZUka8Zoe,Above & Beyond,trance,60,798773
4,10gzBoINW3cLJfZUka8Zoe,Above & Beyond,uplifting trance,60,798773
5,4q3ewBCX7sLwd24euuV69X,Bad Bunny,reggaeton,93,82711140
6,4q3ewBCX7sLwd24euuV69X,Bad Bunny,trap latino,93,82711140
7,4q3ewBCX7sLwd24euuV69X,Bad Bunny,urbano latino,93,82711140
8,25uiPmTg16RbhZWAqwLBy5,Charli xcx,art pop,84,3325618
9,25uiPmTg16RbhZWAqwLBy5,Charli xcx,candy pop,84,3325618


# Albums dataframe

In [18]:
def show_alert(message):
    display(Javascript(f'alert("{message}")'))  # Função para exibir um alerta JavaScript com a mensagem fornecida.

def get_album_data(artist_id, access_token):
    url = f'https://api.spotify.com/v1/artists/{artist_id}/albums?include_groups=album&market=BR'  # Define a URL para obter álbuns de um artista específico pelo ID.
    headers = {'Authorization': f'Bearer {access_token}'}  # Define o cabeçalho de autorização com o token de acesso.
    response = requests.get(url, headers=headers)  # Faz uma requisição GET para a URL com os cabeçalhos definidos.

    if response.status_code != 200:  # Verifica se a resposta não é bem-sucedida (código diferente de 200).
        show_alert(f'Error {response.status_code}: {response.json().get("error", {}).get("message", "Erro ao alimentar a tabela df_album.")}')
        return None  # Exibe um alerta com a mensagem de erro e retorna None.
    return response.json()  # Retorna os dados do álbum em formato JSON.

artist_ids = [
    '10gzBoINW3cLJfZUka8Zoe', '4q3ewBCX7sLwd24euuV69X',
    '25uiPmTg16RbhZWAqwLBy5', '6M2wZ9GZgrQXHCFfjv46we', '0X2BH1fck6amBIoJhDVmmJ'
]  # Lista de IDs de artistas do Spotify.

albums_data = []

for artist_id in artist_ids:
    data = get_album_data(artist_id, access_token)  # Obtém os dados dos álbuns de cada artista usando o ID e o token de acesso.
    if data and 'items' in data:
        for item in data['items']:
            total_tracks = int(item.get('total_tracks', 0))  # Obtém e converte o total de faixas do álbum para inteiro.
            album_id = str(item.get('id', ''))  # Obtém e converte o ID do álbum para string.
            album_name = str(item.get('name', ''))  # Obtém e converte o nome do álbum para string.
            release_date = pd.to_datetime(item.get('release_date', ''), errors='coerce')  # Obtém e converte a data de lançamento para datetime.
            artists_list = [str(artist['name']) for artist in item['artists']]  # Converte os nomes dos artistas para string e cria uma lista.

            albums_data.append({
                'album_id': album_id,
                'album_name': album_name,
                'artist_id': str(artist_id),  # Converte o ID do artista para string.
                'artists': artists_list,  # Armazena a lista de artistas.
                'release_date': release_date,
                'total_tracks': total_tracks
            })  # Adiciona os dados do álbum à lista.

# Cria um DataFrame com os dados dos álbuns.
df_albums = pd.DataFrame(albums_data)

# Explode a coluna 'artists' para que cada linha tenha apenas um artista.
df_albums = df_albums.explode('artists').reset_index(drop=True)

# Salvar o DataFrame em um arquivo CSV
df_albums.to_csv('albums_data.csv', index=False, header=False)  # header=False remove os nomes das colunas do arquivo CSV

df_albums  # Exibe o DataFrame final com os dados dos álbuns.

,album_id,album_name,artist_id,artists,release_date,total_tracks
0,77CZXKYfcv8YU0mslwf0lG,Group Therapy 584 (DJ Mix),10gzBoINW3cLJfZUka8Zoe,Above & Beyond,2024-06-22,27
1,77CZXKYfcv8YU0mslwf0lG,Group Therapy 584 (DJ Mix),10gzBoINW3cLJfZUka8Zoe,Above & Beyond Group Therapy,2024-06-22,27
2,77CZXKYfcv8YU0mslwf0lG,Group Therapy 584 (DJ Mix),10gzBoINW3cLJfZUka8Zoe,Anjunabeats,2024-06-22,27
3,0bKQg6C3XIVC1Q9b8LErDs,Group Therapy 583 (DJ Mix),10gzBoINW3cLJfZUka8Zoe,Above & Beyond Group Therapy,2024-06-15,27
4,0bKQg6C3XIVC1Q9b8LErDs,Group Therapy 583 (DJ Mix),10gzBoINW3cLJfZUka8Zoe,Anjunabeats,2024-06-15,27
...,...,...,...,...,...,...
62,2Dw4fYqDQnxsgoXDdMbqh3,Halcyon Days,0X2BH1fck6amBIoJhDVmmJ,Ellie Goulding,2014-01-01,22
63,1gKVOJVxDTqw9IaCbKaYd4,Halcyon Nights,0X2BH1fck6amBIoJhDVmmJ,Ellie Goulding,2012-10-12,31
64,0UxDUFlte3hkCkvOpmNGgG,Halcyon,0X2BH1fck6amBIoJhDVmmJ,Ellie Goulding,2012-01-01,14
65,0eQwEistImu6rYY5NjNJNm,Lights,0X2BH1fck6amBIoJhDVmmJ,Ellie Goulding,2011-03-08,10


# Top Tracks dataframe

In [19]:
def show_alert(message):
    display(Javascript(f'alert("{message}")'))  # Função para exibir um alerta JavaScript com a mensagem fornecida.

def get_top_track_data(artist_id, access_token):
    url = f'https://api.spotify.com/v1/artists/{artist_id}/top-tracks?market=BR'  # Define a URL para obter as faixas mais populares de um artista específico pelo ID.
    headers = {'Authorization': f'Bearer {access_token}'}  # Define o cabeçalho de autorização com o token de acesso.
    response = requests.get(url, headers=headers)  # Faz uma requisição GET para a URL com os cabeçalhos definidos.

    if response.status_code != 200:  # Verifica se a resposta não é bem-sucedida (código diferente de 200).
        show_alert(f'Error {response.status_code}: {response.json().get("error", {}).get("message", "Erro ao alimentar a tabela df_top_tracks.")}')
        return None  # Exibe um alerta com a mensagem de erro e retorna None.
    return response.json()['tracks']  # Retorna as faixas mais populares do artista em formato JSON.

artist_ids = [
    '10gzBoINW3cLJfZUka8Zoe', '4q3ewBCX7sLwd24euuV69X',
    '25uiPmTg16RbhZWAqwLBy5', '6M2wZ9GZgrQXHCFfjv46we', '0X2BH1fck6amBIoJhDVmmJ'
]  # Lista de IDs de artistas do Spotify.

track_data = []

for artist_id in artist_ids:
    top_tracks = get_top_track_data(artist_id, access_token)  # Obtém as faixas mais populares de cada artista usando o ID e o token de acesso.
    if top_tracks is None:
        continue

    for track in top_tracks:
        track_id = str(track['id'])  # Obtém o ID da faixa e converte para string.
        track_name = str(track['name'])  # Obtém o nome da faixa e converte para string.
        track_duration_ms = int(track['duration_ms'])  # Obtém a duração da faixa em milissegundos e converte para inteiro.
        track_popularity = int(track['popularity'])  # Obtém a popularidade da faixa e converte para inteiro.
        track_release_date = pd.to_datetime(track['album']['release_date'], errors='coerce')  # Obtém a data de lançamento do álbum da faixa e converte para datetime.
        album_id = str(track['album']['id'])  # Obtém o ID do álbum e converte para string.
        album_name = str(track['album']['name'])  # Obtém o nome do álbum e converte para string.

        artists_list = [str(artist['name']) for artist in track['artists']]  # Converte os nomes dos artistas para string e cria uma lista.

        for artist_name in artists_list:
            track_data.append({
                'track_id': track_id,
                'track_name': track_name,
                'album_id': album_id,
                'album_name': album_name,
                'artist_id': str(artist_id),  # Converte o ID do artista para string.
                'artists': artist_name,
                'track_release_date': track_release_date,
                'track_popularity': track_popularity,
                'duration_ms': track_duration_ms
            })  # Adiciona os dados da faixa à lista.

df_top_tracks = pd.DataFrame(track_data)  # Cria um DataFrame com os dados das faixas mais populares, especificando as colunas.

# Salvar o DataFrame em um arquivo CSV
df_top_tracks.to_csv('top_tracks_data.csv', index=False, header=False)  # header=False remove os nomes das colunas do arquivo CSV

df_top_tracks  # Exibe o DataFrame final com os dados das faixas mais populares.

,track_id,track_name,album_id,album_name,artist_id,artists,track_release_date,track_popularity,duration_ms
0,5dXZhZbDHWu6zwXkssFQcs,Blue Monday,743cLwhIfyy5dFZNEF6zOw,Blue Monday,10gzBoINW3cLJfZUka8Zoe,Above & Beyond,2020-01-20,53,252919
1,57ROgavXvK1xMKex1dpbQd,Group Therapy Intro (ABGT583) - Mixed,0bKQg6C3XIVC1Q9b8LErDs,Group Therapy 583 (DJ Mix),10gzBoINW3cLJfZUka8Zoe,Above & Beyond,2024-06-15,36,31248
2,1GHOLip5jcgRIBqd8QUx7N,Believer - Marsh's Guatape Remix,7GGVUzqEZQFwbHsBO3Fzam,Believer (Marsh's Guatape Remix),10gzBoINW3cLJfZUka8Zoe,Above & Beyond,2022-01-06,48,255778
3,1GHOLip5jcgRIBqd8QUx7N,Believer - Marsh's Guatape Remix,7GGVUzqEZQFwbHsBO3Fzam,Believer (Marsh's Guatape Remix),10gzBoINW3cLJfZUka8Zoe,Marsh,2022-01-06,48,255778
4,4rfhAoZjGwraqwX1w47uij,Sun & Moon - Original Mix,2I6sty06i8pXtu175kAfHa,Group Therapy,10gzBoINW3cLJfZUka8Zoe,Above & Beyond,2011-07-04,47,326266
...,...,...,...,...,...,...,...,...,...
73,3X003bO0MMxXEsJJxidiig,How Long Will I Love You,1gKVOJVxDTqw9IaCbKaYd4,Halcyon Nights,0X2BH1fck6amBIoJhDVmmJ,Ellie Goulding,2012-10-12,68,154440
74,5Hp4xFihdOE2dmDzxWcBFb,All By Myself,3lAmnw0gNntYuTltwETnSn,All By Myself,0X2BH1fck6amBIoJhDVmmJ,Alok,2022-10-07,69,171747
75,5Hp4xFihdOE2dmDzxWcBFb,All By Myself,3lAmnw0gNntYuTltwETnSn,All By Myself,0X2BH1fck6amBIoJhDVmmJ,Sigala,2022-10-07,69,171747
76,5Hp4xFihdOE2dmDzxWcBFb,All By Myself,3lAmnw0gNntYuTltwETnSn,All By Myself,0X2BH1fck6amBIoJhDVmmJ,Ellie Goulding,2022-10-07,69,171747
